# Project for "Selected topics of Computer Vision"

### **"PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation"**

##### GitHub Link: https://github.com/charlesq34/pointnet

### PointNet PyTorch implementation: 
##### GitHub Link: https://github.com/intel-isl/Open3D-PointNet

*   Manuel Buchauer
*   Davide De Sclavis




## Data loading


In [5]:
from open3d import *
import numpy as np
from scipy.io import loadmat
import os
from os import listdir
from os.path import isfile, join
import torch.utils.data as data
import time
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from pointnet import PointNetDenseCls
import torch.nn.functional as F
if torch.cuda.is_available():
  import torch.backends.cudnn as cudnn

In [6]:
def load_directory_names(filename):
    
    directory_list = []
    
    file = open(filename, 'r') 
    lines = file.readlines() 

    for line in lines: 
        line_string = line.strip()
        split_string = line_string.split(" ", 2)
        
        directory_list.append(split_string[1])
        
    return directory_list

In [7]:
# get all filenames in a given folder 

def get_all_filenames_from_directory(directory_path):
    onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
    
    onlyfiles.sort()
    
    return onlyfiles

In [8]:
def seperate_file_list_into_pcd_and_label(file_list):
    
    file_list.sort()
    
    pcd_list = [];
    label_list = []
    
    suffix_pcd = ".pcd"
    suffix_mat = ".mat"
    
    for file in file_list:
        
        if(file.endswith(suffix_pcd)):
            pcd_list.append(file)
            
        if(file.endswith(suffix_mat)):
            label_list.append(file)
            
    
    pcd_list.sort()
    label_list.sort()
    
    return pcd_list, label_list

In [9]:
class Dataset(data.Dataset):
    
    def __init__(self, dataset_root, dataset_folders, number_of_points, categories):
        
        self.dataset_root = dataset_root
        # list of folders in root folder
        self.dataset_folders = load_directory_names(dataset_folders)
        self.categories = categories
        self.number_of_points = number_of_points
        
        self.pointcloud_list = []
        self.label_list = []
        
        # load labels and pointclouds into self.pointcloud_list and self.label_list
        self.load_point_clouds()
        
    
    def load_point_clouds(self):
        
        # iterate over all folders
        for folder in self.dataset_folders:

          for j in range(0,len(self.categories)):
              # if first part of folder name equals category name
              if(folder.split('_')[0] == self.categories[j]):
                  # get all filenames in given folder
                  file_list = get_all_filenames_from_directory(self.dataset_root + '/' + folder)
                  
                  pcd_files, label_files = seperate_file_list_into_pcd_and_label(file_list)
                  
                  # iterate over all files in folder, already ensured that pcd_files and label_files have same length
                  for i in range (0, int(len(pcd_files))):
                      
                      # load pointcloud
                      print("Load Data: NR "+ str(i))
                      pcd = open3d.io.read_point_cloud(self.dataset_root + '/' + folder + '/' + pcd_files[i])
                      # load labels
                      labels = loadmat(self.dataset_root + '/' + folder + '/' + label_files[i])
                      
                      self.pointcloud_list.append(pcd)
                      self.label_list.append(labels)  
                    
                    
    def __getitem__(self, index):
        
        labels = self.label_list[index]['data'][0]
        points = np.array(self.pointcloud_list[index].points)
        
        # randomly choose number of points form point cloud and labels 
        choice = np.random.choice(len(points), self.number_of_points, replace=True)

        points = points[choice, :]
        labels = labels[choice]
        
        points = points.astype(np.float32)
        
        torch_labels = torch.from_numpy(labels)
        torch_points = torch.from_numpy(points)
        choice=0
        return torch_points, torch_labels 
    
    
    def __len__(self):
        return len(self.pointcloud_list)

### Load training pcd data

In [11]:
d = Dataset('./dataset', './tool_categories.txt', 2500, ['bowl','cup','hammer','knife','mallet','mug','scissors','spoon','turner'])


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Load Data: NR 4
Load Data: NR 5
Load Data: NR 6
Load Data: NR 7
Load Data: NR 8
Load Data: NR 9
Load Data: NR 10
Load Data: NR 11
Load Data: NR 12
Load Data: NR 13
Load Data: NR 14
Load Data: NR 15
Load Data: NR 16
Load Data: NR 17
Load Data: NR 18
Load Data: NR 19
Load Data: NR 20
Load Data: NR 21
Load Data: NR 22
Load Data: NR 23
Load Data: NR 24
Load Data: NR 25
Load Data: NR 26
Load Data: NR 27
Load Data: NR 28
Load Data: NR 29
Load Data: NR 30
Load Data: NR 31
Load Data: NR 32
Load Data: NR 33
Load Data: NR 34
Load Data: NR 35
Load Data: NR 36
Load Data: NR 37
Load Data: NR 38
Load Data: NR 39
Load Data: NR 40
Load Data: NR 41
Load Data: NR 42
Load Data: NR 43
Load Data: NR 44
Load Data: NR 45
Load Data: NR 46
Load Data: NR 47
Load Data: NR 48
Load Data: NR 49
Load Data: NR 50
Load Data: NR 51
Load Data: NR 52
Load Data: NR 53
Load Data: NR 54
Load Data: NR 55
Load Data: NR 56
Load Data: NR 57
Load Data: NR 58
Load

In [13]:
test_dataset = Dataset('./dataset', './tool_categories_test.txt', 2500, ['bowl','cup','hammer','knife','mallet','mug','scissors','spoon','turner'])

Load Data: NR 0
Load Data: NR 1
Load Data: NR 2
Load Data: NR 3
Load Data: NR 4
Load Data: NR 5
Load Data: NR 6
Load Data: NR 7
Load Data: NR 8
Load Data: NR 9
Load Data: NR 10
Load Data: NR 11
Load Data: NR 12
Load Data: NR 13
Load Data: NR 14
Load Data: NR 15
Load Data: NR 16
Load Data: NR 17
Load Data: NR 18
Load Data: NR 19
Load Data: NR 20
Load Data: NR 21
Load Data: NR 22
Load Data: NR 23
Load Data: NR 24
Load Data: NR 25
Load Data: NR 26
Load Data: NR 27
Load Data: NR 28
Load Data: NR 29
Load Data: NR 30
Load Data: NR 31
Load Data: NR 32
Load Data: NR 33
Load Data: NR 34
Load Data: NR 35
Load Data: NR 36
Load Data: NR 37
Load Data: NR 38
Load Data: NR 39
Load Data: NR 40
Load Data: NR 41
Load Data: NR 42
Load Data: NR 43
Load Data: NR 44
Load Data: NR 45
Load Data: NR 46
Load Data: NR 47
Load Data: NR 48
Load Data: NR 49
Load Data: NR 50
Load Data: NR 51
Load Data: NR 52
Load Data: NR 53
Load Data: NR 54
Load Data: NR 55
Load Data: NR 56
Load Data: NR 57
Load Data: NR 58
Load Da

In [16]:
dataloader = torch.utils.data.DataLoader(d, 32, shuffle=True, num_workers=4)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle = True)

In [17]:
# look if cuda is available
torch.cuda.is_available()

True

## Training 

In [ ]:
blue = lambda x:'\033[94m' + x + '\033[0m'

num_classes = 7
print('classes', num_classes)

classifier = PointNetDenseCls(k = num_classes)
#classifier.load_state_dict(torch.load(''))


optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
if torch.cuda.is_available():
  classifier.cuda()


for epoch in range(30):
  for i, data in enumerate(dataloader, 0):
    #hotfix because of error files with len of 1
    if(len(data[0])==32):
      points, target = data
      target = target.type(torch.LongTensor)
      points, target = Variable(points), Variable(target)
      points = points.transpose(2,1)
      if torch.cuda.is_available():
        print('Cuda')
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        print(len(points))
        pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1,1)[:,0] - 1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' %(epoch, i, 32, loss.item(), correct.item()/float(32 * 2500)))
      
        if i % 10 == 0:
          print('here')
          j, data = next(enumerate(testdataloader, 0))
          points, target = data
          target = target.type(torch.LongTensor)
          points, target = Variable(points), Variable(target)
          points = points.transpose(2,1)
          if torch.cuda.is_available():
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1,1)[:,0] - 1
          
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' %(epoch, i, 32, blue('test'), loss.item(), correct.item()/float(32 * 2500)))

  torch.save(classifier.state_dict(), '%s/seg_model_%d.pth' % ('seg', epoch))

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
32
[11: 144/32] train loss: 0.155044 accuracy: 0.938975
Cuda
32
[11: 145/32] train loss: 0.223266 accuracy: 0.927075
Cuda
32
[11: 146/32] train loss: 0.185546 accuracy: 0.934550
Cuda
32
[11: 147/32] train loss: 0.144095 accuracy: 0.949150
Cuda
32
[11: 148/32] train loss: 0.450300 accuracy: 0.853662
Cuda
32
[11: 149/32] train loss: 0.161576 accuracy: 0.945275
Cuda
32
[11: 150/32] train loss: 0.196078 accuracy: 0.923800
here
[11: 150/32] test loss: 1.172309 accuracy: 0.751925
Cuda
32
[11: 151/32] train loss: 0.449378 accuracy: 0.810025
Cuda
32
[11: 152/32] train loss: 0.285147 accuracy: 0.889212
Cuda
32
[11: 153/32] train loss: 0.199370 accuracy: 0.933850
Cuda
32
[11: 154/32] train loss: 0.134217 accuracy: 0.956612
Cuda
32
[11: 155/32] train loss: 0.242973 accuracy: 0.903563
Cuda
32
[11: 156/32] train loss: 0.239985 accuracy: 0.903825
Cuda
32
[11: 157/32] train loss: 0.184032 accuracy: 0.932987
Cuda
32
[11: 158/32] train 